# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [40]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')

In [3]:
engine.table_names()

['InsertTableName']

In [4]:
df = pd.read_sql('SELECT * FROM "InsertTableName"', engine)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26215 entries, 0 to 26214
Data columns (total 39 columns):
message                   26215 non-null object
original                  10170 non-null object
genre                     26215 non-null object
related                   26215 non-null int64
request                   26215 non-null int64
offer                     26215 non-null int64
aid_related               26215 non-null int64
medical_help              26215 non-null int64
medical_products          26215 non-null int64
search_and_rescue         26215 non-null int64
security                  26215 non-null int64
military                  26215 non-null int64
child_alone               26215 non-null int64
water                     26215 non-null int64
food                      26215 non-null int64
shelter                   26215 non-null int64
clothing                  26215 non-null int64
money                     26215 non-null int64
missing_people            26215 non-null i

In [8]:
X = df[['message','genre']]
Y = df.drop(columns=['message','original','genre'])

In [9]:
X = X.merge(pd.get_dummies(X['genre']),left_index=True, right_index=True).drop(columns=['genre'])

In [14]:
X['message'] = X['message'].str.lower()

In [28]:
X

,message,direct,news,social
0,weather update - a cold front from cuba that c...,1,0,0
1,is the hurricane over or is it not over,1,0,0
2,looking for someone but no name,1,0,0
3,un reports leogane 80-90 destroyed. only hospi...,1,0,0
4,"says: west side of haiti, rest of the country ...",1,0,0
5,information about the national palace-,1,0,0
6,storm at sacred heart of jesus,1,0,0
7,"please, we need tents and water. we are in sil...",1,0,0
8,"i would like to receive the messages, thank you",1,0,0
9,i am in croix-des-bouquets. we have health iss...,1,0,0


In [32]:
X['message'].apply(lambda x: re.sub('[^\w\s]','',x))

0        weather update  a cold front from cuba that co...
1                  is the hurricane over or is it not over
2                          looking for someone but no name
3        un reports leogane 8090 destroyed only hospita...
4        says west side of haiti rest of the country to...
5                    information about the national palace
6                           storm at sacred heart of jesus
7        please we need tents and water we are in silo ...
8           i would like to receive the messages thank you
9        i am in croixdesbouquets we have health issues...
10       theres nothing to eat and water we starving an...
11       i am in petionville i need more information re...
12       i am in thomassin number 32 in the area named ...
13       lets do it together need food in delma 75 in d...
14       more information on the 4636 number in order f...
15       a comitee in delmas 19 rue  street  janvier im...
16       we need food and water in klecin 12 we are dyi.

### 2. Write a tokenization function to process your text data

In [38]:
def tokenize(text):
    tokens = word_tokenize(re.sub('[^\w\s]',' ',text))
    lem = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lem.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
       
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [41]:

pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', MultiOutputClassifier(RandomForestClassifier())
])

SyntaxError: invalid syntax (<ipython-input-41-3715d36c6023>, line 14)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.